In [1]:
from math import ceil
import json
from os import listdir

import plotly.colors as pcolors
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import ipywidgets as ipw
from IPython.display import display, clear_output
import numpy as np

from simulation import run_simulation
from graph_create import reduce_providers_simplest, powerlaw_dist_time, watts_strogatz, erdos_renyi, configuration_model
import util

# To getossible parameters for each model
import run_model_comp as rmc

In [2]:
init_notebook_mode(connected=True)

In [3]:
def plot_utils(plot_data, plot_title, show_indiv):
    fig = go.Figure()

    colors = pcolors.qualitative.Vivid
    color_idx = 0
    for model, (model_utils, model_avg_diam) in plot_data.items():
    
        #Add averages trace
        p_vals = [ p * 100 / 4 for (p, uts) in model_utils ]
        ut_vals = [ np.average(uts) for (p, uts) in model_utils ]
        fig.add_trace(go.Scatter(x=p_vals,
            y=ut_vals, mode='lines+markers', hoverinfo='skip',
            marker=dict(color=colors[color_idx]),
            name=f'avg social welfare {model} (avg diam {model_avg_diam})'))

        #Add individual points trace
        if show_indiv:
            indiv_p_vals = []
            indiv_ut_vals = []
            for (p, uts) in model_utils:
                indiv_ut_vals.extend(uts)
                indiv_p_vals.extend([p * 100 / 4] * len(uts))
            fig.add_trace(go.Scatter(x=indiv_p_vals,
                y=indiv_ut_vals, mode='markers', hoverinfo='skip', opacity=0.2,
                marker=dict(color=colors[color_idx]), name='social welfare per p'))
            
        color_idx += 1

    fig.layout.update(title=plot_title, showlegend=True,
        xaxis=dict(title='p'), yaxis=dict(title='Average social welfare (normalized)'),
        plot_bgcolor='rgba(0,0,0,0)')

    iplot(fig)

In [16]:
# Load data
overall_data = {}
for data_file in listdir('sim_data'):
    if data_file.split('.')[1] != 'json':
        continue
    data_fp = 'sim_data/{0}'.format(data_file)
    with open(data_fp, 'r') as df:
        file_contents = json.loads(df.read())
        
        try:
            print(file_contents['er']['64.0'].keys())
        except KeyError:
            pass
        
        # NEEDS TO BE UPDATED IF FORMATTING CHANGED
        for model, model_dict in file_contents.items():
            if model not in overall_data:
                overall_data[model] = {}
            for r_val, r_dict in model_dict.items():
                if r_val not in overall_data[model]:
                    overall_data[model][r_val] = {}
                for param, param_data in r_dict.items():
                    if model == 'er' and r_val == '64.0':
                        print(param)
                    plaw = param_data['plaw']
                    if param not in overall_data[model][r_val]:
                        overall_data[model][r_val][param] = {plaw : param_data}
                    overall_data[model][r_val][param].update({plaw : param_data})
                        
print(overall_data['er']['2.0'].keys())
print(overall_data['er']['64.0']['0.034'].keys())

dict_keys(['0.003'])
0.003
dict_keys(['0.034'])
0.034
dict_keys(['0.046'])
0.046
dict_keys(['0.01'])
0.01
dict_keys(['0.003'])
0.003
dict_keys(['0.01'])
0.01
dict_keys(['0.046'])
0.046
dict_keys(['0.034'])
0.034
dict_keys(['0.034', '0.007', '0.046'])
dict_keys([False, True])


In [5]:
# Get possible parameter values

psbl_r_vals = rmc.r_vals
psbl_ws_beta_vals = rmc.beta_vals
psbl_er_ep_vals = [str(round(ep, 3)) for ep in rmc.ep_vals]

In [6]:
class ModelCompPlotter:
    def __init__(self):
        
        self.model_type_widget = ipw.SelectMultiple(
                                    options=['ws', 'ws-cm', 'er'],
                                    description='Model to show',
                                    disabled=False,
                                    orientation='horizontal',
                                    continuous_update=False,
                                    readout=True)
        self.r_val_widget = ipw.Dropdown(
                                    options=psbl_r_vals,
                                    description='r value of simulation',
                                    disabled=False,
                                    orientation='horizontal',
                                    continuous_update=False,
                                    readout=True)
        self.ws_beta_widget = ipw.Dropdown(
                                    options=psbl_ws_beta_vals,
                                    description='Beta value for WS models',
                                    disabled=False,
                                    orientation='horizontal',
                                    continuous_update=False,
                                    readout=True)
        self.er_ep_widget = ipw.Dropdown(
                                    options=psbl_er_ep_vals,
                                    description='Edge probabilities for ER models',
                                    disabled=False,
                                    orientation='horizontal',
                                    continuous_update=False,
                                    readout=True)
        self.show_indiv_widget = ipw.Checkbox(
                                    value=True,
                                    description='Show individual runs',
                                    disabled=False,
                                    continuous_update=False,
                                    orientation='horizontal',
                                    readout=True)
        self.plaw_widget = ipw.Checkbox(
                                    value=False,
                                    description='Powerlaw distributed resources',
                                    disabled=False,
                                    continuous_update=False,
                                    orientation='horizontal',
                                    readout=True)
        
        #Update button
        self.plot_button = ipw.Button(
                                    description='Get plot',
                                    disabled=False,
                                    button_style='',
                                    tooltip='Get plot for simulation with given params')
        self.plot_button.on_click(self.render_visuals)

        #Widgets to render, in order
        self.render_set = [self.model_type_widget, self.r_val_widget, 
                           self.ws_beta_widget, self.er_ep_widget,
                           self.show_indiv_widget, self.plaw_widget,
                           self.plot_button]
        self.render_visuals(None)
        
    def render_visuals(self, button):
        #Reset output
        clear_output()
        
        #Render widgets
        for ritem in self.render_set:
            display(ritem)

        r_val = str(self.r_val_widget.value)
        models = self.model_type_widget.value
        
        plaw = self.plaw_widget.value
        
        plot_data = {}
        
        for model in models:
            param_val = None
            if model in ['ws', 'ws-cm']:
                param_val = self.ws_beta_widget.value
            elif model in ['er']:
                param_val = self.er_ep_widget.value
            param_val = str(param_val)
        
            sim_data = overall_data[model][r_val][param_val][plaw]
            sim_utils = sim_data['utils']
            avg_diam = sim_data['avg_diam']
            
            plot_data[model] = (sim_utils, avg_diam)
        
        show_indiv = self.show_indiv_widget.value
        plot_title = 'Social utility (normalized) vs p'
        if plaw:
            plot_title += ' (Powerlaw distributed resources)'
        
        plot_utils(plot_data, plot_title, show_indiv)

In [7]:
ModelCompPlotter()

SelectMultiple(description='Model to show', index=(2,), options=('ws', 'ws-cm', 'er'), value=('er',))

Dropdown(description='r value of simulation', index=5, options=(2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0), value…

Dropdown(description='Beta value for WS models', options=(0.1,), value=0.1)

Dropdown(description='Edge probabilities for ER models', index=5, options=('0.0', '0.003', '0.007', '0.01', '0…

Checkbox(value=True, description='Show individual runs')

Checkbox(value=False, description='Powerlaw distributed resources')

Button(description='Get plot', style=ButtonStyle(), tooltip='Get plot for simulation with given params')

KeyError: False